# Homework 6: 

**DUE:** 5pm EST, April 9, 2020



### **Conceptual:** Short answer questions. Be concise

---

1. Let us consider the "curse of dimensionality" (i.e., when the ratio of predictor variables, p, to observations, n, is high) in the context of kNN. Answer the following:

<br>

(a) Suppose that we have a set of observations, each with measurements on p = 1 feature, X. We assume that X is uniformly (evenly) distributed between 0 and 1. Associated with each observation is a response value. Suppose that we wish to predict a test observation’s response using only observations that are within 10% of the range of X closest to that test observation. For instance, in order to predict the response for a test observation with X = 0.6, we will use observations in the range [0.55,0.65]. On average, what fraction of the available observations will we use to make the prediction?

**Answer:**



(b) Suppose that we have a set of observations, each with measurements on p = 2 features, X1 and X2. We assume that (X1,X2) are uniformly distributed between 0 and 1. We wish to predict a test observation’s response using only observations that are within 10% of the range of X1 and within 10% of the range of X2 closest to that test observation. For instance, in order to predict the response for a test observation with X1 = 0.6 and X2 = 0.35, we will use observations in the range [0.55, 0.65] for X1 and in the range [0.3, 0.4] for X2. On average, what fraction of the available observations will we use to make the prediction?

**Answer:**


(c) Now suppose that we have a set of observations on p = 100 features. Again the observations are uniformly distributed on each feature, and again each feature ranges in value from 0 to 1. We wish to predict a test observation’s response using observations within the 10% of each feature’s range that is closest to that test observation. What fraction of the available observations will we use to make the prediction?

**Answer:**




(c)  Using your answers to parts (a)–(c), argue that a drawback of KNN when p is large is that there are very few training observations “near” any given test observation.

**Answer:**



---

2. Explain how k-fold cross-validation is implemented. What are the advantages and disadvantages of k-fold cross-validation relative to LOOCV?


**Answer:**


---

3. Differentiate the bootstrap from a permutation test. Describe each and when is it appropriate to each.

**Answer:**



---

### **Applied:** Show your code & plots

We will use both datasets from the HCP dataset for these problems. Include conceptual answers to questions as comments in code cells. You should use the tidyverse and class libraries for this assignment, as well as the data table for the HCP data set.

4. Use kNN to build a classifier that for predicting Gender using the unadjusted Flanker task performance (Flanker_unadj). 

(a) Split the data set up into 1206 training observations and 200 test observations. Clasffiy using kNN with k = 5. Show your classification accuracy as a predicted vs. observed table, and report the mean classification accuracy.

In [1]:
# -------------------------------
# Question 4
# -------------------------------


# (a)


(b) Now repeat your analysis using a range of k's, from 1-100. Plot the performance of the classifier (i.e., accuracy) for each value of k. Which value gives you the highest accuracy and what is the accuracy for that best k?

In [2]:
# (b)


5. Use bootstrapping to estimate the confidence on the regression coefficients for a logistic regression model using Flanker_unadj scores to predict Gender. Use 5000 iterations off the bootstrap. Report whether any of the coefficients are statistically significant using the standard error estimates off of the bootstrap.

In [3]:
# -------------------------------
# Question 5
# -------------------------------
